# Ejercicio validación

Para el siguiente ejemplo, se solicita importar los datos del dataset `student-mat.csv` y realizar el proceso de validación utilizando los siguientes mecanismos 

- Validación Hold-out
- Validación Cruzada

>Note: Para las predicciones utilizar [sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). 

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

## Importar datos

In [2]:
df = pd.read_csv('student-mat.csv', sep = ';') #TODO
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [3]:
# Transformaciones

categorical_columns = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

# Aplicar la codificación one-hot a las columnas categóricas
df2_encoded = pd.get_dummies(df, columns = categorical_columns, drop_first = True)
df2_encoded.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,True,False,True,False,False,False,True,True,False,False
1,17,1,1,1,2,0,5,3,3,1,...,False,False,False,True,False,False,False,True,True,False
2,15,1,1,1,2,3,4,3,2,2,...,True,False,True,False,True,False,True,True,True,False
3,15,4,2,1,3,0,3,2,2,1,...,True,False,False,True,True,True,True,True,True,True
4,16,3,3,1,2,0,4,3,2,1,...,False,False,False,True,True,False,True,True,False,False


In [4]:
df2_encoded

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,True,False,True,False,False,False,True,True,False,False
1,17,1,1,1,2,0,5,3,3,1,...,False,False,False,True,False,False,False,True,True,False
2,15,1,1,1,2,3,4,3,2,2,...,True,False,True,False,True,False,True,True,True,False
3,15,4,2,1,3,0,3,2,2,1,...,True,False,False,True,True,True,True,True,True,True
4,16,3,3,1,2,0,4,3,2,1,...,False,False,False,True,True,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,...,False,True,False,True,True,False,True,True,False,False
391,17,3,1,2,1,0,2,4,5,3,...,True,False,False,False,False,False,False,True,True,False
392,21,1,1,1,1,3,5,5,3,3,...,False,True,False,False,False,False,False,True,False,False
393,18,3,2,3,1,0,4,4,1,3,...,True,False,False,False,False,False,False,True,True,False


## Validación Hold-out

In [5]:
y = df2_encoded['G3']

In [6]:
#Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_training, X_test, y_training, y_test = train_test_split(df2_encoded.drop('G3', axis=1), y, test_size = 0.2, random_state = 42)
valores_test, ocur_test = np.unique(y_test, return_counts=True)

print('Test: ', 'clases:', valores_test)
print('\n')
print('ocurrencias: ', ocur_test)

Test:  clases: [ 0  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


ocurrencias:  [ 5  4  6  1  6  5 11  5  5  5  6 10  4  3  1  2]


In [7]:
# Estandarizar las características de entrenamiento y de test
standardizer = StandardScaler()
X_training = standardizer.fit_transform(X_training)
X_test = standardizer.transform(X_test)

In [8]:
# Validación: hold-out split 80-20%. # Partición interna
X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, test_size=0.2, random_state=42) #TODO

valores_train, ocur_train = np.unique(y_train, return_counts=True)
print('Entrenamiento: ', ' clases:', valores_train)
print('\n')
print('ocurrencias:', ocur_train)
print('\n')


valores_val, ocur_val = np.unique(y_val, return_counts=True)

print('Test: ', 'clases:', valores_val)
print('\n')
print('ocurrencias: ', ocur_val)

Entrenamiento:   clases: [ 0  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


ocurrencias: [27  1  3  6  5 17 16 34 30 22 23 18 21 12  3 11  2  1]


Test:  clases: [ 0  6  7  8  9 10 11 12 13 14 15 19]


ocurrencias:  [ 6  3  3  9  7 11 12  4  3  3  2  1]


In [9]:
# Construcción del objeto que contiene el algoritmo de aprendizaje.
# Utilizamos un algoritmo de regresión lineal
lm = LinearRegression() #TODO
lm


LinearRegression()

In [10]:
lm.fit(X_train,y_train) # TODO - Entrenamos modelo

val_accuracy = lm.score(X_val, y_val) # TODO - Evaluamos modelo en validación
print('Exactitud en validación ', np.round(val_accuracy*100, 2), '%')

test_accuracy = lm.score(X_test, y_test) # TODO - Evaluamos modelo en test
print('Exactitud en test ', np.round(test_accuracy*100, 2), '%')

Exactitud en validación  82.99 %
Exactitud en test  71.74 %


## Validación Cruzada

In [11]:
# Test: hold-out split 80-20%. PARTICIÓN EXTERNA
X_train, X_test, y_train, y_test = train_test_split(df2_encoded.drop('G3', axis=1), y, test_size = 0.2, random_state = 42) #TODO

valores_test, ocur_test = np.unique(y_test, return_counts=True)
print('Test: ', 'clases:', valores_test, ' ocurrencias: ', ocur_test)

valores_train, ocur_train = np.unique(y_train, return_counts=True)
print('Entrenamiento: ', ' clases:', valores_train, '  ocurrencias:', ocur_train)

Test:  clases: [ 0  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]  ocurrencias:  [ 5  4  6  1  6  5 11  5  5  5  6 10  4  3  1  2]
Entrenamiento:   clases: [ 0  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]   ocurrencias: [33  1  3  9  8 26 23 45 42 26 26 21 23 12  3 11  3  1]


In [12]:
# Construcción del objeto que contiene el algoritmo de aprendizaje.
# Utilizamos un algoritmo de regresión lineal
lm = LinearRegression() #TODO

In [13]:
# Validación, entrenamiento y evaluación del algoritmo de aprendizaje.
# en "cv = KFold(n_splits=5)" estamos haciendo un cross-validation INTERNO!

"""
En results se hace la validación cruzada
"""

results = cross_val_score(lm, 
                          X_train, 
                          y_train, 
                          cv = KFold(n_splits=5, shuffle = True, random_state = 42)) #TODO

print("Resultados por bolsa: ", results)
print("Accuracy (media +/- desv.): %0.4f +/- %0.4f" % (results.mean(), results.std()))

Resultados por bolsa:  [0.82990804 0.78811931 0.76169781 0.84982299 0.78971299]
Accuracy (media +/- desv.): 0.8039 +/- 0.0317


In [14]:
# Una vez entrenado y validado el modelo para seleccionar los mejores hyperparameters, 
# utilizamos todos los datos de "train" y "val" para entrenar el modelo definitivo

lm = lm.fit(X_train, y_train) # TODO - Entrenamiento
test_results = lm.score(X_test, y_test) # TODO - Evaluación en test
print('Exactitud en test: ', test_results*100, '%')
print('\n')
# También podemos extraer las predicciones, en lugar de directamente la accuracy
y_pred = lm.predict(X_test) #TODO
y=pd.Series(y_pred)
print('Predicciones:\n', y)
print('\n')
print('Etiquetas reales:\n', y_test)

Exactitud en test:  72.41341236974021 %


Predicciones:
 0      6.001607
1     11.528478
2      2.866437
3      8.796631
4      8.553106
        ...    
74    10.156740
75     6.275788
76     7.686656
77    12.409067
78     4.338174
Length: 79, dtype: float64


Etiquetas reales:
 78     10
371    12
248     5
55     10
390     9
       ..
364    12
82      6
114     9
3      15
18      5
Name: G3, Length: 79, dtype: int64
